# Trust Region Policy Optimization (TRPO)

In [ ]:
!apt-get install -y xvfb

!pip install gym==0.23.1 \
    pytorch-lightning==1.6 \
    pyvirtualdisplay

!pip install -U brax==0.0.12 jax==0.3.14 jaxlib==0.3.14+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

import warnings 
warnings.filterwarnings('ignore')

#### Setup virtual display

In [1]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

#### Import the necessary code libraries

In [2]:
import copy
import torch
import random
import gym
import matplotlib
import functools
import itertools
import math

import numpy as np
import matplotlib.pyplot as plt

import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW, Optimizer

from torch.distributions import Normal, kl_divergence #for calculating KL divergence from the samples

from pytorch_lightning import LightningModule, Trainer

import brax.v1
from brax.v1 import envs
from brax.v1.envs import to_torch
from brax.v1.io import html

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

v = torch.ones(1, device='cuda')

In [3]:
@torch.no_grad()
def create_video(env, episode_length, policy=None):
  qp_array = []
  state = env.reset()
  for i in range(episode_length):
    if policy:
      loc, scale = policy(state)
      sample = torch.normal(loc, scale)
      action = torch.tanh(sample)
    else:
      action = env.action_space.sample()
    state, _, _, _ = env.step(action)
    qp_array.append(env.unwrapped._state.qp)
  return HTML(html.render(env.unwrapped._env.sys, qp_array))


@torch.no_grad()
def test_agent(env, episode_length, policy, episodes=10):

  ep_returns = []
  for ep in range(episodes):
    state = env.reset()
    done = False
    ep_ret = 0.0

    while not done:
      loc, scale = policy(state)
      sample = torch.normal(loc, scale)
      action = torch.tanh(sample)   
      state, reward, done, info = env.step(action)
      ep_ret += reward.item()

    ep_returns.append(ep_ret)

  return sum(ep_returns) / episodes  

#### Create the policy

In [4]:
class GradientPolicy(nn.Module):

  def __init__(self, in_features, out_dims, hidden_size=128):
    super().__init__()
    self.fc1 = nn.Linear(in_features, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc_mu = nn.Linear(hidden_size, out_dims)
    self.fc_std = nn.Linear(hidden_size, out_dims)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    loc = self.fc_mu(x)
    loc = torch.tanh(loc)
    scale = self.fc_std(x)
    scale = F.softplus(scale) + 0.001
    return loc, scale

#### Create the value network

In [5]:
class ValueNet(nn.Module):

  def __init__(self, in_features, hidden_size=128):
    super().__init__()
    self.fc1 = nn.Linear(in_features, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

#### Create the environment

In [6]:
class RunningMeanStd:
    # https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Parallel_algorithm
    def __init__(self, epsilon=1e-4, shape=()):
        self.mean = torch.zeros(shape, dtype=torch.float32).to(device)
        self.var = torch.ones(shape, dtype=torch.float32).to(device)
        self.count = epsilon

    def update(self, x):
        batch_mean = torch.mean(x, dim=0)
        batch_var = torch.var(x, dim=0)
        batch_count = x.shape[0]
        self.update_from_moments(batch_mean, batch_var, batch_count)

    def update_from_moments(self, batch_mean, batch_var, batch_count):
        self.mean, self.var, self.count = update_mean_var_count_from_moments(
            self.mean, self.var, self.count, batch_mean, batch_var, batch_count
        )


def update_mean_var_count_from_moments(
    mean, var, count, batch_mean, batch_var, batch_count
):
    delta = batch_mean - mean
    tot_count = count + batch_count

    new_mean = mean + delta * batch_count / tot_count
    m_a = var * count
    m_b = batch_var * batch_count
    M2 = m_a + m_b + torch.square(delta) * count * batch_count / tot_count
    new_var = M2 / tot_count
    new_count = tot_count

    return new_mean, new_var, new_count


class NormalizeObservation(gym.core.Wrapper):

    def __init__(self, env, epsilon=1e-8):
        super().__init__(env)
        self.num_envs = getattr(env, "num_envs", 1)
        self.obs_rms = RunningMeanStd(shape=self.observation_space.shape[-1])
        self.epsilon = epsilon

    def step(self, action):
        obs, rews, dones, infos = self.env.step(action)
        obs = self.normalize(obs)
        return obs, rews, dones, infos

    def reset(self, **kwargs):
        return_info = kwargs.get("return_info", False)
        if return_info:
            obs, info = self.env.reset(**kwargs)
        else:
            obs = self.env.reset(**kwargs)
        obs = self.normalize(obs)
        if not return_info:
            return obs
        else:
            return obs, info

    def normalize(self, obs):
        self.obs_rms.update(obs)
        return (obs - self.obs_rms.mean) / torch.sqrt(self.obs_rms.var + self.epsilon)


In [7]:
entry_point = functools.partial(envs.create_gym_env, env_name='ant')
gym.register('brax-ant-v0', entry_point=entry_point)

In [8]:
env = gym.make("brax-ant-v0", episode_length=1000)
env = to_torch.JaxToTorchWrapper(env, device=device)
create_video(env, 1000)

/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'jaxlib.xla_extension.ArrayImpl'>`
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarni

In [9]:
def create_env(env_name, num_envs=256, episode_length=1000):
  env = gym.make(env_name, batch_size=num_envs, episode_length=episode_length)
  env = to_torch.JaxToTorchWrapper(env, device=device)
  env = NormalizeObservation(env)
  return env

In [10]:
env = create_env('brax-ant-v0', num_envs=10)
obs = env.reset()
print("Num envs: ", obs.shape[0], "Obs dimentions: ", obs.shape[1])

/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (10, 87)
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(


Num envs:  10 Obs dimentions:  87


In [11]:
env.action_space

Box(-1.0, 1.0, (10, 8), float32)

In [ ]:
obs, reward, done, info = env.step(env.action_space.sample())

In [ ]:
info.keys()

#### TRPO optimizer

In [12]:
# Adapted from: https://github.com/rlworkgroup/garage/blob/master/src/garage/torch/optimizers/conjugate_gradient_optimizer.py


# Copyright (c) 2019 Reinforcement Learning Working Group

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


def unflatten_tensors(flattened, tensor_shapes):
  flattened = flattened.cpu()
  tensor_sizes = list(map(np.prod, tensor_shapes))
  indices = np.cumsum(tensor_sizes)[:-1]
  return [
      np.reshape(pair[0], pair[1]).to(device)
      for pair in zip(np.split(flattened, indices), tensor_shapes)
  ]


def _build_hessian_vector_product(func, params, reg_coeff=1e-5):
    param_shapes = [p.shape or torch.Size([1]) for p in params]
    f = func()
    f_grads = torch.autograd.grad(f, params, create_graph=True)

    def _eval(vector):
      unflatten_vector = unflatten_tensors(vector, param_shapes)
      

      assert len(f_grads) == len(unflatten_vector)
      grad_vector_product = torch.sum(
          torch.stack(
              [torch.sum(g * x) for g, x in zip(f_grads, unflatten_vector)]))

      hvp = list(
          torch.autograd.grad(grad_vector_product, params,
                              retain_graph=True))
      for i, (hx, p) in enumerate(zip(hvp, params)):
          if hx is None:
              hvp[i] = torch.zeros_like(p)

      flat_output = torch.cat([h.reshape(-1) for h in hvp])
      return flat_output + reg_coeff * vector

    return _eval


def _conjugate_gradient(f_Ax, b, cg_iters, residual_tol=1e-10):#this direction allows to compute the direction of our gradient step 
    #without having to build to the hessian matrix entirely
    #remember that hessain matrix is the second derivative of the KL divergence with respect to the parameters of our policy 
    p = b.clone()
    r = b.clone()
    x = torch.zeros_like(b)
    rdotr = torch.dot(r, r)

    for _ in range(cg_iters):
        z = f_Ax(p)
        v = rdotr / torch.dot(p, z)
        x += v * p
        r -= v * z
        newrdotr = torch.dot(r, r)
        mu = newrdotr / rdotr
        p = r + mu * p

        rdotr = newrdotr
        if rdotr < residual_tol:
            break
    return x


class ConjugateGradientOptimizer(Optimizer):

    def __init__(self, params, max_constraint_value, cg_iters=10, max_backtracks=15,
                 backtrack_ratio=0.8, hvp_reg_coeff=1e-5, accept_violation=False):
      
        super().__init__(params, {})
        self._max_constraint_value = max_constraint_value
        self._cg_iters = cg_iters
        self._max_backtracks = max_backtracks
        self._backtrack_ratio = backtrack_ratio
        self._hvp_reg_coeff = hvp_reg_coeff
        self._accept_violation = accept_violation


    def step(self, closure):
      f_loss, f_constraint = closure()
      #f_loss computes the value of our surrogate function 
      #f_constraint will compute the value of our constraint
      #the reason why we pass it as functions instead of computed values is because of the iterative process where we try to find 
      #the maximum step size that will try to satsify the KL divergence constraint
      #since we will have to compute these values several times, we are passing functions to compute them
      params = []
      grads = []
      for group in self.param_groups:
          for p in group['params']:
              if p.grad is not None:
                  params.append(p)
                  grads.append(p.grad.reshape(-1))

      flat_loss_grads = torch.cat(grads)
      f_Ax = _build_hessian_vector_product(f_constraint, params, self._hvp_reg_coeff)
      step_dir = _conjugate_gradient(f_Ax, flat_loss_grads, self._cg_iters)

      step_dir[step_dir.ne(step_dir)] = 0.

      step_size = np.sqrt(2.0 * self._max_constraint_value * (1. / (torch.dot(step_dir, f_Ax(step_dir)) + 1e-8)).cpu())

      if np.isnan(step_size):
          step_size = 1.

      descent_step = step_size * step_dir
      self._backtracking_line_search(params, descent_step, f_loss, f_constraint)


    def _backtracking_line_search(self, params, descent_step, f_loss, f_constraint):#this is the function that will try several step sizes until one of them lands us
        #in out trust region
        prev_params = [p.clone() for p in params]
        ratio_list = self._backtrack_ratio**np.arange(self._max_backtracks)
        loss_before = f_loss()

        param_shapes = [p.shape or torch.Size([1]) for p in params]
        descent_step = unflatten_tensors(descent_step, param_shapes)
        assert len(descent_step) == len(params)

        for ratio in ratio_list:
            for step, prev_param, param in zip(descent_step, prev_params, params):
                step = ratio * step
                new_param = prev_param.data - step
                param.data = new_param.data

            loss = f_loss()
            constraint_val = f_constraint()
            if (loss < loss_before and constraint_val <= self._max_constraint_value):
                break

#### Create the dataset

In [13]:
class RLDataset(IterableDataset):

  def __init__(self, env, policy, value_net, samples_per_epoch, gamma, lamb):#we delete the repeat parameter because the TRPO algorithm 
    #already takes care of taking the largest step allowed in our trust region

    self.samples_per_epoch = samples_per_epoch
    self.gamma = gamma
    self.lamb = lamb
    self.env = env
    self.policy = policy
    self.value_net = value_net
    self.obs = self.env.reset()
  
  @torch.no_grad()
  def __iter__(self):
    transitions = []
    for step in range(self.samples_per_epoch):
      loc, scale = self.policy(self.obs)
      action = torch.normal(loc, scale)
      next_obs, reward, done, info = self.env.step(action)
      transitions.append((self.obs, loc, scale, action, reward, done, next_obs))
      self.obs = next_obs
  
    transitions = map(torch.stack, zip(*transitions))
    obs_b, loc_b, scale_b, action_b, reward_b, done_b, next_obs_b = transitions
    reward_b = reward_b.unsqueeze(dim=-1)
    done_b = done_b.unsqueeze(dim=-1)

    values_b = self.value_net(obs_b)
    next_values_b = self.value_net(next_obs_b)

    td_error_b = reward_b + (1 - done_b) * self.gamma * next_values_b - values_b

    running_gae = torch.zeros((self.env.num_envs, 1), dtype=torch.float32, device=device)
    gae_b = torch.zeros_like(td_error_b)

    for row in range(self.samples_per_epoch - 1, -1, -1):
      running_gae = td_error_b[row] + (1 - done_b[row]) * self.gamma * self.lamb * running_gae
      gae_b[row] = running_gae

    target_b = gae_b + values_b

    num_samples = self.samples_per_epoch * self.env.num_envs
    reshape_fn = lambda x: x.view(num_samples, -1)
    batch = [obs_b, loc_b, scale_b, action_b, reward_b, gae_b, target_b]

    obs_b, loc_b, scale_b, action_b, reward_b, gae_b, target_b = map(reshape_fn, batch)

    idx = list(range(num_samples))
    random.shuffle(idx)

    for i in idx:
      yield obs_b[i], loc_b[i], scale_b[i], action_b[i], reward_b[i], gae_b[i], target_b[i]

#### Create TRPO with generalized advantage estimation (GAE)

In [14]:
class TRPO(LightningModule):

  def __init__(self, env_name, num_envs=2048, episode_length=1000,
               batch_size=2048, hidden_size=256, samples_per_epoch=20,
               value_lr=1e-3, gamma=0.99, lamb=0.95,  kl_limit=0.25, 
               v_optim=AdamW, pi_optim=ConjugateGradientOptimizer):
    
    super().__init__()

    self.env = create_env(env_name, num_envs=num_envs, episode_length=episode_length)
    test_env = gym.make(env_name, episode_length=episode_length)
    test_env = to_torch.JaxToTorchWrapper(test_env, device=device)
    self.test_env = NormalizeObservation(test_env)
    self.test_env.obs_rms = self.env.obs_rms

    obs_size = self.env.observation_space.shape[1]
    action_dims = self.env.action_space.shape[1]

    self.policy = GradientPolicy(obs_size, action_dims, hidden_size)
    self.value_net = ValueNet(obs_size, hidden_size)
    self.target_value_net = copy.deepcopy(self.value_net)

    self.pi_optim = ConjugateGradientOptimizer(self.policy.parameters(), max_constraint_value=kl_limit)

    self.dataset = RLDataset(self.env, self.policy, self.target_value_net,
                             samples_per_epoch, gamma, lamb)

    self.save_hyperparameters()
    self.videos = []

    self.automatic_optimization = False

  def configure_optimizers(self):
    value_opt = self.hparams.v_optim(self.value_net.parameters(), lr=self.hparams.value_lr)
    pi_optim = ConjugateGradientOptimizer(self.policy.parameters(), max_constraint_value=self.hparams.kl_limit)#this is the difference
    return [value_opt, pi_optim]

  def train_dataloader(self):
    return DataLoader(dataset=self.dataset, batch_size=self.hparams.batch_size)    

  def training_step(self, batch, batch_idx):
    obs_b, loc_b, scale_b, action_b, reward_b, gae_b, target_b = batch
    v_optim, pi_optim = self.optimizers()

    state_values = self.value_net(obs_b)
    v_loss = F.smooth_l1_loss(state_values, target_b)
    self.log("episode/Value Loss", v_loss)

    v_optim.zero_grad()
    self.manual_backward(v_loss)
    v_optim.step()

    new_loc, new_scale = self.policy(obs_b)
    dist = Normal(new_loc, new_scale)
    log_prob = dist.log_prob(action_b).sum(dim=-1, keepdim=True)

    prev_dist = Normal(loc_b, scale_b)
    prev_log_prob = prev_dist.log_prob(action_b).sum(dim=-1, keepdim=True)
    #this time our optimiser will have to call loss function several times in order to take the optimisation step because remember 
    #the algorithm that finds the solution to the optimsation problem will try severeal step sizes until it reaches one that lands inside our trust region
    #to facilitate that, we are going to create a function called loss_fun()
    def loss_fn():
      loss = - torch.exp(log_prob - prev_log_prob) * gae_b
      return loss.mean()#scalar value needed

    def constraint_fn():
      constraint = kl_divergence(prev_dist, dist).sum(dim=-1)
      return constraint.mean()#scalar value needed
    
    closure = lambda: (loss_fn, constraint_fn)

    loss = loss_fn()

    pi_optim.zero_grad()
    self.manual_backward(loss, retain_graph=True)#as we need to compute the gradient of the same computation graph several times, we need to save the computation_graph
    pi_optim.step(closure)

    self.log("episode/Policy Loss", loss)
    self.log("episode/Reward", reward_b.mean())


  def on_train_epoch_end(self):
    self.target_value_net.load_state_dict(self.value_net.state_dict())

    if self.current_epoch % 10 == 0:
      average_return = test_agent(self.test_env, self.hparams.episode_length, self.policy, episodes=1)
      self.log("episode/Average Return", average_return)

    if self.current_epoch % 50 == 0:
      video = create_video(self.test_env, self.hparams.episode_length, policy=self.policy)
      self.videos.append(video)

#### Purge logs and run the visualization tool (Tensorboard)

In [ ]:
# Start tensorboard.
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

#### Train the policy

In [15]:
algo = TRPO('brax-ant-v0')

trainer = Trainer(
                # gpus=num_gpus, 
                max_epochs=1000)

trainer.fit(algo)

/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (2048, 87)
  logger.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-07-09 16:06:04.949525: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_

Training: |          | 0/? [00:00<?, ?it/s]

/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'jaxlib.xla_extension.ArrayImpl'>`
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: Detected Keyboar

In [ ]:
algo.videos[-1]